<a href="https://colab.research.google.com/github/aj225patel/python-fundamentals/blob/main/tasks/task4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import pprint
import time

def get_data_types(spreadsheet_url):
    # Read the Google Spreadsheet into a pandas DataFrame
    df = pd.read_csv(spreadsheet_url)

    # Get the data types for each column
    data_types = df.dtypes

    return data_types

if __name__ == '__main__':
  # Replace 'your_google_spreadsheet_url' with the actual URL of your Google Spreadsheet
  google_spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1pOVI9P_uzGg-zVJRgv0ONDtUd3F5qOy3zGo3-YBiql0/edit?usp=sharing'
  sheet_id = '1pOVI9P_uzGg-zVJRgv0ONDtUd3F5qOy3zGo3-YBiql0'
  updated_url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv'

  start_time = time.time()
  result = get_data_types(updated_url)
  end_time = time.time()
  print("Data Types for each column:")

  result = result.to_dict()
  pp = pprint.PrettyPrinter(indent=4)
  pp.pprint(result)
  print(f'total time elapsed: {end_time - start_time}s')

Data Types for each column:
{   'Activity 1': dtype('O'),
    'Activity 2': dtype('O'),
    'Activity 3': dtype('float64'),
    'Activity 4': dtype('float64'),
    'Admission Current Status': dtype('float64'),
    'Admission Year': dtype('O'),
    'Admitted NPS': dtype('float64'),
    'Applicant Channel': dtype('O'),
    'Applicant ID': dtype('int64'),
    'Applicant Source': dtype('O'),
    'Application Complete': dtype('O'),
    'Application Complete Month': dtype('O'),
    'Application Complete Week': dtype('int64'),
    'Application Processed': dtype('O'),
    'Application Processed Month': dtype('O'),
    'Application Processed Week': dtype('int64'),
    'Application Started': dtype('O'),
    'Application Started Week': dtype('int64'),
    'Application Type': dtype('float64'),
    'Applicaton Started Month': dtype('O'),
    'Apply Grade': dtype('int64'),
    'Attribute 1': dtype('float64'),
    'Attribute 2': dtype('O'),
    'Attribute 3': dtype('float64'),
    'Attribute 4': dtyp

In [3]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
# import pprint
import time

# Path to the JSON file containing the service account credentials
json_keyfile_path = '/content/secret/iconic-range-412912-258f129777ff.json'

# Google Sheet URL
sheet_url = 'https://docs.google.com/spreadsheets/d/1pOVI9P_uzGg-zVJRgv0ONDtUd3F5qOy3zGo3-YBiql0/edit?usp=sharing'


# Function to authenticate and open the Google Sheet
def open_google_sheet(json_keyfile_path, sheet_url):
    scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

    credentials = ServiceAccountCredentials.from_json_keyfile_name(json_keyfile_path, scope)
    gc = gspread.authorize(credentials)

    # Open the Google Sheet by URL
    spreadsheet = gc.open_by_url(sheet_url)
    return spreadsheet

# Function to load Google Sheet data into a Pandas DataFrame
def load_google_sheet_into_dataframe(json_keyfile_path, sheet_url):
    spreadsheet = open_google_sheet(json_keyfile_path, sheet_url)

    # Select the first (default) sheet
    worksheet = spreadsheet.get_worksheet(0)

    # Get all values from the sheet
    data = worksheet.get_all_values()
    # print(data[:5])

    # Create a Pandas DataFrame
    df = pd.DataFrame(data[1:], columns=data[0])

    return df

def preprocess_numeric_value(value):
    # If the value contains commas, remove them before attempting conversion
    return str(value).replace(',', '')

def infer_datatype(value):
    try:
        # Try converting to integer
        return int(preprocess_numeric_value(value))
    except ValueError:
        try:
            # Try converting to float
            return float(value)
        except ValueError:
            try:
                # Try converting to datetime
                return pd.to_datetime(value)
            except ValueError:
                # If all conversions fail, check for Yes/No and treat as bool
                if str(value).lower() in ['yes', 'no']:
                    return bool(value.lower() == 'yes')
                # If all conversions fail, consider it as a string
                return str(value)

def generate_datatype_schema(df, type_dict):
    schema = {}
    for column in df.columns:
        # Use the infer_datatype function to handle different types
        filtered_df = df.loc[df[column] != '', column]

        # print(df[column].apply(lambda x: type(x)).value_counts())

        if len(filtered_df) != 0:
          first_valid_element = filtered_df.iloc[0]
        else:
          first_valid_element = df[column].iloc[0]

        data_type = str(type(infer_datatype(first_valid_element)).__name__)

        if(type_dict.get(data_type)):
          column_data_type = type_dict[data_type]
        else:
          column_data_type = data_type

        schema[column] = column_data_type
    return schema


def main():
  type_dict = {'str': 'text', 'NaTType': 'text', 'int': 'number', 'Timestamp': 'date' }

  start_time = time.time()
  # Fetch data from Google Sheet
  data_df = load_google_sheet_into_dataframe(json_keyfile_path, sheet_url)

  # Generate datatype schema
  schema = generate_datatype_schema(data_df, type_dict)

  # Print the generated schema
  print("Generated Datatype Schema:")
  for column, data_type in schema.items():
      print(f"{column}: {data_type}")

  end_time = time.time()

  print(f'Total elapsed time: {end_time - start_time}')

if __name__ == '__main__':
  main()

Generated Datatype Schema:
User ID: text
BB ID: number
Applicant ID: number
Student ID: number
Ver ID: number
Rav ID: number
Household ID: number
Nav ID: text
Date Snapshot: text
Source: text
Full Name: text
Full Name - Concat: text
First Name: text
Last Name: text
Prefered Name: text
Gender: text
Date of Birth: date
Race: text
Ethnicity: text
Current Grade: number
Apply Grade: number
Application Type: text
Is International: bool
Is FA: text
Is Legacy: text
Is Sibling: text
Is Faculty: text
Admission Year: text
Entering Year: text
Tuition Amount: text
Financal Aid Amount: text
Non Need Amount: text
Tuition Due: text
Applicant Source: text
Applicant Channel: text
Inquiry Submitted: date
Application Started: date
Application Complete: date
Application Processed: date
Inquiry Submitted Week: number
Inquiry Submitted Month: text
Application Started Week: number
Applicaton Started Month: text
Application Complete Week: number
Application Complete Month: text
Application Processed Week: numb